In [1]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from object_detection.utils import visualization_utils as viz_utils
import cv2
import random

In [8]:
wd = r'C:\Users\bcper\Documents\GitHub\models\workspace\data'

wd = r'C:\Users\bcper\Documents\GitHub\xView_data_utilities'
record_file = r'C:\Users\bcper\Documents\GitHub\models\workspace\data\xview_train_cleaned_complete.record'

#record_file = r'C:\Users\bcper\Desktop\racoon\test.record'

os.chdir(wd)

In [12]:
raw_image_dataset = tf.data.TFRecordDataset(record_file)

image_feature_description = {
            'image/height': tf.io.FixedLenFeature([], tf.int64),
            'image/width': tf.io.FixedLenFeature([], tf.int64),
            'image/encoded': tf.io.FixedLenFeature([], tf.string),
            'image/format': tf.io.FixedLenFeature([], tf.string),
            'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
            'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
            'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

def _parse_image_function(raw_image_dataset):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(raw_image_dataset, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

In [13]:
class_labels = 'Class Labels.txt'
path = r'C:\Users\bcper\Documents\GitHub\geoJSON interpreter'
labelpath = path + '\\' + class_labels

#read in the class labels
class_dict = {}
file = open(labelpath, "r")

#create a class label dictionary
for line in file:
    key, value = line.split(':')
    class_dict[int(key)] = value.strip()

In [14]:
#to get the number of records in the dataset
sum(1 for _ in tf.data.TFRecordDataset(record_file))

44460

In [7]:
window_name = 'Image'
for record in parsed_image_dataset.take(5):
    image = tf.image.decode_jpeg(record['image/encoded'])
    label = record['image/object/class/label'].values
    
    image = image.numpy()
    
    size_y = image.shape[0]
    size_x = image.shape[1]
    
    xmin = record['image/object/bbox/xmin'].values
    xmax = record['image/object/bbox/xmax'].values
    ymin = record['image/object/bbox/ymin'].values
    ymax = record['image/object/bbox/ymax'].values
    '''
    print(xmin)
    print(xmax)
    print(ymin)
    print(ymax)
    '''
    labels = label.numpy()
    xmins = xmin.numpy()
    xmaxs = xmax.numpy()
    ymins = ymin.numpy()
    ymaxs = ymax.numpy()
    
    color_dict = {}
    type_set = np.unique(labels)
    
    for type_id in type_set:
        color_dict[type_id] = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
    
    # font needed for text
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    for label in range(len(labels)):
        id_type = labels[label]
        
        topleft = (int(xmins[label]*size_x), int(ymins[label]*size_y))
        bottomright = (int(xmaxs[label]*size_x), int(ymaxs[label]*size_y))
        
        image = cv2.rectangle(image, topleft, bottomright, color=color_dict[id_type], thickness=1)
        #image = cv2.putText(image, ('ID: ' + str(id_type)), topleft)
        image = cv2.putText(image, class_dict[id_type], topleft, font, fontScale=0.5, color=color_dict[id_type], thickness=2)
        
    # Display the image 
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

tf.Tensor(
[0.818 0.916 0.806 0.98  0.746 0.596 0.234 0.    0.528 0.464 0.844 0.944
 0.256 0.17  0.004 0.262 0.116 0.65  0.712 0.522 0.42  0.232 0.    0.232
 0.012 0.862 0.164 0.384 0.484 0.092 0.394 0.772 0.294 0.356], shape=(34,), dtype=float32)
tf.Tensor(
[0.86  0.98  0.858 1.    0.81  0.658 0.358 0.038 0.612 0.534 0.946 1.
 0.318 0.246 0.194 0.328 0.198 0.726 0.848 0.574 0.458 0.348 0.026 0.28
 0.166 0.898 0.288 0.452 0.548 0.186 0.484 0.812 0.414 0.494], shape=(34,), dtype=float32)
tf.Tensor(
[0.464 0.412 0.418 0.084 0.032 0.008 0.    0.406 0.864 0.9   0.09  0.14
 0.556 0.862 0.238 0.958 0.068 0.    0.068 0.012 0.81  0.35  0.194 0.132
 0.    0.42  0.    0.316 0.33  0.532 0.962 0.442 0.034 0.09 ], shape=(34,), dtype=float32)
tf.Tensor(
[0.486 0.45  0.458 0.128 0.072 0.126 0.136 0.564 0.982 0.982 0.142 0.208
 0.612 0.976 0.404 1.    0.15  0.154 0.144 0.06  0.836 0.464 0.262 0.19
 0.026 0.49  0.066 0.388 0.374 0.626 1.    0.494 0.176 0.24 ], shape=(34,), dtype=float32)
tf.Tensor([0.2